![title](img/decisionmaking.png)

# Modeling decision making

Begin by the beginning: import the modules

In [13]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import scipy.optimize
import statsmodels.stats
import scipy.stats
from tqdm.notebook import tqdm

EPS = np.finfo(np.float).eps

## 1. Task model & Cognitive models

### 1.1 Task definition

**Goal**

- Check the adequacy with the question you want to reply to
- Select adequate characteristics (i.e., enough time-steps) 

**Example: Bandit task**

![bandit](img/multiarmedbandit.jpg)

Parameters: 
* Number of option ($N$)
* Distribution of probability over these options ($\{p_{reward}(i)\}_{i\in N}$)
* Number of trials ($T$)

In [14]:
N = 2
P = np.array([0.5, 0.75])
T = 500

N_SUBJECTS = 30

### 1.2 Models definition

**Goal**

- Select/Define which aspect(s) you want to model
- Model alternatives

<img src="img/coin.png" width=200>

**Random selection**

* Decision rule:

\begin{equation}
p_{choice}(i) \sim \text{Uniform}
\end{equation}

<img src="img/wsls.png" width=200>

**Win-Stay-Loose-Switch (Noisy variant)**

* Decision rule ($\epsilon$)

\begin{equation}
p_{choice}(i) = 
\begin{cases}
1−\epsilon+\epsilon / N &\text{if } r_{t−1} =1 \wedge c_{t−1} = i,\\
\epsilon / N &\text{if } r_{t−1} =1 \wedge c_{t−1} \neq i,\\
\dfrac{1−\epsilon}{N-1} + \epsilon / N & \text {otherwise.}\\
\end{cases}
\end{equation}


<img src="img/rw_illustration.png" width=200>

**Rescola-Wagner**

* Decision rule ($\beta$)

\begin{equation}
p_{choice}(i) = \dfrac{\exp (Q(i)/\beta)}{\sum_{j \in N} \exp (Q(j)/\beta)} 
\end{equation}

* Updating rule ($\alpha$)

\begin{equation}
Q^{t+1}(i) = Q^t(i) + \alpha(s - Q^t(i))
\end{equation}


![rw](img/status_quo.jpg)

**Rescola-Wagner-Choice-Kernel

* Decision rule ($\beta_q$, $\beta_c$)

\begin{equation}
p_{choice}(i) = \dfrac{\exp (Q(i)/\beta_q+ C(i)/\beta_c)}{\sum_{j \in N} 
\exp (Q(j)/\beta_q+ C(j)/\beta_c)} 
\end{equation}

* Updating rule ($\alpha$)

\begin{align}
Q^{t+1}(i) &= Q^t(i) + \alpha(s - Q^t(i)) \\
C^{t+1}(i) &= C^t(i) + \alpha(a^t(i) - C^t(i)) \\
\end{align}
where $a^t(i) = 1$ if option $i$ selected at time $t$ else 0.

# 3. Simulate 

Determine the (log) likelihood of your model:

\begin{equation}
\log p(D \mid M, \theta) = \sum_{d \in D} \log p(d \mid M, \theta)
\end{equation}


\begin{equation}
\text{BIC} = - 2 \log p(D\mid {\widehat {\theta }},M) + k \log(T)
\end{equation}

In [15]:
def bic(ll, k, n_iteration):
    return -2 *ll + k * np.log(n_iteration)